In [1]:
import Tasks0 
top100songs = Tasks0.top100songs

In [2]:
import requests
import requests_cache
requests_cache.install_cache("cache")
from bs4 import BeautifulSoup
import re
import string
import pandas as pd
test = top100songs.loc[9] 
def processString(s, type):
    s = re.sub("['?:!.,;&/]+", " ", s).lower().strip()
    if(type != "Artist"):
        s = re.sub("-", "",s)
    s = re.sub("\s+", "-", s)
    return(s)

def getLyrics(song, newUrl = ''):
    artist = song['Artist']
    title = song['Title']
    #Remove punctuation and change to lower case.
    artist = processString(artist, "Artist")
    title = processString(title," Title")
    lyric_url = "http://www.songlyrics.com/"+artist+"/"+title+"-lyrics/"
    if newUrl != '':
        lyric_url = newUrl
#    print(lyric_url)
    soup = BeautifulSoup((requests.get(lyric_url)).text, "html.parser")
    tmp = soup.find_all('p', {"id": "songLyricsDiv"})
    lyrics = tmp[0].get_text()
    if bool(re.search(pattern= "(We do not have)|(we have no)", string = lyrics)):
        lyrics = ""
        lyric_url = ""
    return({'Url':lyric_url, 'Lyric': lyrics})


In [110]:
result = [getLyrics(row) for idx, row in top100songs.iterrows()]
result = pd.DataFrame(result)

In [111]:
#Locate songs that couldn't be found using function
missing =result[result['Url'] == ""].index
top100songs.loc[missing]

,Artist,Title,Year
53,Elton John,Candle in the Wind 1997 / Something About the ...,1997
60,Paul McCartney and Michael Jackson,"Say, Say, Say",1983
80,Percy Faith,The Theme from 'A Summer Place',1960
83,Jewel,You Were Meant for Me / Foolish Games,1997


In [112]:
notFoundUrls = ["http://www.songlyrics.com/elton-john-billy-joel/candle-in-the-wind-lyrics/", "http://www.songlyrics.com/percy-faith/theme-from-a-summer-place-lyrics/","http://www.songlyrics.com/jewel-feat-kelly-clarkson/foolish-games-lyrics/"]

In [113]:
result.loc[53,'Lyric':'Url'] = getLyrics(top100songs.loc[53], newUrl = notFoundUrls[0])
result.loc[80,'Lyric':'Url'] = getLyrics(top100songs.loc[80], newUrl = notFoundUrls[1])
result.loc[83,'Lyric':'Url'] = getLyrics(top100songs.loc[83], newUrl = notFoundUrls[2])

In [114]:
song = top100songs.loc[60]
artist = song['Artist']
title = song['Title']
lyric_url = "http://www.metrolyrics.com/say-say-say-lyrics-paul-mccartney.html"
artist = processString(artist, "Artist")
title = processString(title," Title")
soup = BeautifulSoup((requests.get(lyric_url)).text, "html.parser")
tmp = soup.find_all('div', {"id": "lyrics-body-text"})
lyrics = tmp[0].get_text()
result.loc[60,'Lyric': 'Url'] = {'Lyric': lyrics, 'Url': lyric_url}

In [115]:
result = pd.concat([top100songs.reset_index(drop=True), result], axis = 1)
result.loc[53,'Title'] = "Candle in the Wind 1997"
result.loc[83, 'Title'] = "Foolish Games"

In [116]:
client_id = '37b4f8e242b54a67a4b8d4807b35d4aa'
client_secret = '9cb223d471b64dac9f1dc48f3b62593f'
#client_id = "Enter your client_id from spotify"
#client_secret = "Enter your client_id"

def spotify_authorize(client_id, client_secret):
    grant_type = 'client_credentials'

    #Request based on Client Credentials Flow from https://developer.spotify.com/web-api/authorization-guide/

    #Request body parameter: grant_type Value: Required. Set it to client_credentials
    body_params = {'grant_type' : grant_type}

    url='https://accounts.spotify.com/api/token'

    response=requests.post(url, data=body_params, auth = (client_id, client_secret)) 
    return(response.json())

token = spotify_authorize(client_id, client_secret)['access_token']

In [117]:
def spotify_search(term, search_type = "artist", verbose = False):
    url = "https://api.spotify.com/v1/search"
    response = requests.get(url, params = {
        "q": term,
        "type": search_type
    })
    response.raise_for_status() # check for errors
    if verbose:
        print response.url
    
    return response.json() # parse JSON

In [123]:
def spotify_audio_features(trackid, token, verbose = False):
    url = "https://api.spotify.com/v1/audio-features/"+trackid
    headers = {"Authorization":"Bearer" + ' ' + str(token)}
    response = requests.get(url, headers = headers)
    response.raise_for_status() # check for errors
    if verbose:
        print response.url

    return response.json() # parse JSON

In [135]:
def getSpotifyID(search, token):
    """
    Search for song title and artist combination on spotify and return id, duration, and other audio characteristcs
    """
    id = search['tracks']['items'][0]['id']
    length = search['tracks']['items'][0]['duration_ms']
    audio_features = spotify_audio_features(id, token)
    result = {'spotifyID': id, 
              'songLength': audio_features['duration_ms'],
              'acousticness': audio_features['acousticness'],
              'danceability': audio_features['danceability'],
              'energy': audio_features['energy'], 
              'instrumentalness': audio_features['instrumentalness'],
              'key': audio_features['key'],
              'liveness': audio_features['liveness'],
              'loudness': audio_features['loudness'],
              'mode': audio_features['mode'],
              'speechiness': audio_features['speechiness'],
              'tempo': audio_features['tempo'],
             'valence': audio_features['valence']}
    return result

In [136]:
idSearch = list()
for index, row in result.iterrows():
    search_result = spotify_search(row['Title'] + ' ' + row['Artist'], "track")
    idSearch.append(getSpotifyID(search_result, token))
    

In [140]:
final_data = pd.concat([result.reset_index(drop=True), pd.DataFrame(idSearch)], axis = 1)

In [141]:
final_data

,Artist,Title,Year,Lyric,Url,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,songLength,speechiness,spotifyID,tempo
0,Usher,You Make Me Wanna...,1997,This is what you do\nThis is what you do\nThis...,http://www.songlyrics.com/usher/you-make-me-wa...,0.03590,0.761,0.6390,0.000000,5,0.0945,-7.577,0,219120,0.0539,47TqCCnEliDp8NRDyIQoQq,164.088
1,fun. feat. Janelle Monae,We Are Young,2012,"Give me a second, I\nI need to get my story st...",http://www.songlyrics.com/fun-feat-janelle-mon...,0.02000,0.378,0.6380,0.000077,10,0.0849,-5.576,1,250627,0.0750,7a86XRg84qjasly9f6bPSD,184.086
2,Emotions,Best of My Love,1977,Every night I'm lyin' in bed\nHoldin' you clos...,http://www.songlyrics.com/emotions/best-of-my-...,0.45000,0.764,0.5700,0.362000,0,0.0812,-11.361,1,219933,0.0348,6UjlpJhUqOQqv0hrb38IDI,115.376
3,Maroon 5,One More Night,2012,You and I go hard at each other like we're goi...,http://www.songlyrics.com/maroon-5/one-more-ni...,0.05630,0.710,0.8000,0.000000,5,0.0834,-3.439,0,219467,0.0339,4XNrMwGx1SqP01sqkGTDmo,93.010
4,Phil Collins,Another Day in Paradise,1989,"She calls out to the man on the street\n""Sir, ...",http://www.songlyrics.com/phil-collins/another...,0.73600,0.780,0.5650,0.002690,3,0.0630,-7.320,0,322133,0.0306,1NCuYqMc8hKMb4cpNTcJbD,101.967
5,B.J. Thomas,Raindrops Keep Fallin' on My Head,1970,Raindrops are falling on my head\nand just lik...,http://www.songlyrics.com/b-j-thomas/raindrops...,0.81300,0.705,0.2610,0.018700,5,0.0876,-18.934,1,182667,0.0495,1KkwByi6c9dFFmUTcgMODw,108.716
6,Puff Daddy & Faith Evans feat. 112,I'll Be Missing You,1997,Yeah... this right here (tell me why)\nGoes ou...,http://www.songlyrics.com/puff-daddy-faith-eva...,0.11100,0.833,0.6190,0.000021,7,0.1590,-5.595,1,306560,0.0538,1yy2DlSDtEt90d54rPDPXz,109.920
7,John Cougar,Hurts So Good,1982,When I was a young boy\nSaid put away those yo...,http://www.songlyrics.com/john-cougar/hurts-so...,0.04750,0.812,0.7240,0.000045,9,0.1090,-5.321,1,218960,0.0355,67eX1ovaHyVPUinMHeUtIM,125.437
8,Roberta Flack,Killing Me Softly With His Song,1973,Strumming my pain with his fingers\nSinging my...,http://www.songlyrics.com/roberta-flack/killin...,0.72700,0.477,0.3810,0.000000,5,0.0615,-10.237,0,286920,0.0342,3gsCAGsWr6pUm1Vy7CPPob,121.568
9,Elvis Presley,Are You Lonesome To-Night?,1960,Are you lonesome tonight? Do you miss me tonig...,http://www.songlyrics.com/elvis-presley/are-yo...,0.81800,0.542,0.4660,0.769000,0,0.1740,-7.557,1,178200,0.0277,4XFLwpF2cgrZs0ctFume45,78.883
